In [1]:
%pip install --upgrade pip
%pip install python-dotenv
%pip install openai --upgrade
%pip install time

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/d4/55/90db48d85f7689ec6f81c0db0622d704306c5284850383c090e6c7195a5c/pip-24.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
    --------------------------------------- 0.0/1.8 MB 660.6 kB/s eta 0:00:03
   ---------- ----------------------------- 0.5/1.8 MB 5.2 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 14.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Attempting uninstall: openai
    Found existing installation: openai 1.37.0
    Uninstalling openai-1.37.0:
      Successfully uninstalled openai-1.37.0
Note: you m

ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [2]:
import os
import time
import json
from dotenv import load_dotenv, find_dotenv
from openai import AzureOpenAI

In [5]:
# Load environment variables from .env.local if it exists, otherwise fallback to .env
load_dotenv('.env.local')
load_dotenv()

False

In [6]:
client = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_KEY"],
    api_version=os.environ["AZURE_OPENAI_VERSION"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
)

In [7]:
# Create an assistant
assistant = client.beta.assistants.create(
    name="Financial Analyst Assistant2",
    instructions="You are an expert financial analyst. Use you knowledge base to answer questions about audited financial statements.",
    tools=[{"type": "file_search"}],
    model=os.environ["AZURE_OPENAI_MODEL"] #You must replace this value with the deployment name for your model.
)

print(assistant)

Assistant(id='asst_bL1G4RKwYkP5Gg41Do61pfed', created_at=1722542107, description=None, instructions='You are an expert financial analyst. Use you knowledge base to answer questions about audited financial statements.', metadata={}, model='gpt-4o', name='Financial Analyst Assistant2', object='assistant', tools=[FileSearchTool(type='file_search', file_search=None)], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=[])), top_p=1.0)


In [8]:
# Add Vector Store
# Create a vector store caled "Financial Statements"
vector_store = client.beta.vector_stores.create(name="Financial Statements")
 
# Ready the files for upload to OpenAI
file_paths = ["files/MSFT_2023_Annual_Report.docx"]
file_streams = [open(path, "rb") for path in file_paths]
 
# # Use the upload and poll SDK helper to upload the files, add them to the vector store,
# # and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(vector_store)
print(file_batch.status)
print(file_batch.file_counts)

VectorStore(id='vs_YUqO4MTKizpPkSCnqCgUhE6V', created_at=1722542110, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=0, total=0), last_active_at=1722542110, metadata={}, name='Financial Statements', object='vector_store', status='completed', usage_bytes=0, expires_after=None, expires_at=None)
completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [9]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

print(assistant)

Assistant(id='asst_bL1G4RKwYkP5Gg41Do61pfed', created_at=1722542107, description=None, instructions='You are an expert financial analyst. Use you knowledge base to answer questions about audited financial statements.', metadata={}, model='gpt-4o', name='Financial Analyst Assistant2', object='assistant', tools=[FileSearchTool(type='file_search', file_search=None)], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_YUqO4MTKizpPkSCnqCgUhE6V'])), top_p=1.0)


In [10]:
# Create a thread
thread = client.beta.threads.create()

print(thread)

Thread(id='thread_jvdj7uY6Vy3XAfFMGrYrT6W0', created_at=1722542133, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [11]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user", ## actually the only role available - stay tuned !
    content="How many company shares were outstanding last quarter?"
)

print(message)

Message(id='msg_iWrq5J00hbTy3hm71mx6Ej1e', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='How many company shares were outstanding last quarter?'), type='text')], created_at=1722542138, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_jvdj7uY6Vy3XAfFMGrYrT6W0')


In [12]:
# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [13]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status

print(run)

Run(id='run_A65HKJ6hswpHH7kiHWY8gaZf', assistant_id='asst_bL1G4RKwYkP5Gg41Do61pfed', cancelled_at=None, completed_at=None, created_at=1722542241, expires_at=1722542841, failed_at=None, incomplete_details=None, instructions='You are an expert financial analyst. Use you knowledge base to answer questions about audited financial statements.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=None, required_action=None, response_format='auto', started_at=1722542241, status='in_progress', thread_id='thread_jvdj7uY6Vy3XAfFMGrYrT6W0', tool_choice='auto', tools=[FileSearchTool(type='file_search', file_search=None)], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


In [14]:
# Wait till the assistant has responded
while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    status = run.status

messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [15]:
print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_z909fJc0OvU5JLclnslVWT1v",
      "assistant_id": "asst_bL1G4RKwYkP5Gg41Do61pfed",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [
              {
                "end_index": 130,
                "file_citation": {
                  "file_id": "assistant-9LEQ1Vr46yGvrUEijeiSa8qg"
                },
                "start_index": 118,
                "text": "【4:0†source】",
                "type": "file_citation"
              }
            ],
            "value": "As of the last quarter, which ended on June 30, 2023, the company had 7,432 million shares of common stock outstanding【4:0†source】."
          },
          "type": "text"
        }
      ],
      "created_at": 1722542243,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_A65HKJ6hswpHH7kiHWY8ga

In [16]:
assistant_files = client.files.list()

print(assistant_files)

SyncPage[FileObject](data=[FileObject(id='assistant-9LEQ1Vr46yGvrUEijeiSa8qg', bytes=1451962, created_at=1722542111, filename='MSFT_2023_Annual_Report.docx', object='file', purpose='assistants', status='processed', status_details=None)], object='list')


In [17]:
all_files = client.beta.vector_stores.list()

print(all_files)

SyncCursorPage[VectorStore](data=[VectorStore(id='vs_YUqO4MTKizpPkSCnqCgUhE6V', created_at=1722542110, file_counts=FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1), last_active_at=1722542241, metadata={}, name='Financial Statements', object='vector_store', status='completed', usage_bytes=493395, expires_after=None, expires_at=None)], object='list', first_id='vs_YUqO4MTKizpPkSCnqCgUhE6V', last_id='vs_YUqO4MTKizpPkSCnqCgUhE6V', has_more=False)
